# Source
This Notebook was mostly taken from https://github.com/nicknochnack/TFODCourse. (1.08.2021)

It was adapted to train multiple models and customized to use the prepared datasets.

# 0. Setup Paths

In [2]:
import os
import ipywidgets as widgets

objectChooser = widgets.Dropdown(
    options=['eye', 'iris', 'pupil', 'unity_eyes'],
    value='pupil',
    description='Train for:',
    disabled=False,
)
modelChooser = widgets.Dropdown(
    options=['coco'],
    value='coco',
    description='Weights:',
    disabled=False,
)
display(objectChooser)
display(modelChooser)

Dropdown(description='Train for:', index=2, options=('eye', 'iris', 'pupil', 'unity_eyes'), value='pupil')

Dropdown(description='Weights:', options=('coco',), value='coco')

In [3]:
CUSTOM_MODEL_NAME = 'ssd_mobnet_' + modelChooser.value + "_"+ objectChooser.value 
LABEL_MAP_NAME = 'eyes_label_map.pbtxt'
LABEL_TXT_NAME = 'labels.txt'
META_JSON_DEFAULT = 'meta.json'
META_JSON_CUSTOM = 'meta' + modelChooser.value + '.json'

if modelChooser.value == 'coco':
    PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
    PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
    
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join(objectChooser.value),
    'MODEL_PATH': os.path.join(objectChooser.value, 'models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join(objectChooser.value, 'models', CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join(objectChooser.value, 'models', CUSTOM_MODEL_NAME, 'export'), 
    'TFLITE_PATH':os.path.join(objectChooser.value, 'models', CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
 }
files = {
    'PIPELINE_CONFIG':os.path.join(objectChooser.value, 'models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME),
    'LABELTXT': os.path.join(paths['ANNOTATION_PATH'], LABEL_TXT_NAME),
    'META_DEFAULT': os.path.join('Tensorflow', META_JSON_DEFAULT),   
    'META_CUSTOM': os.path.join(paths['ANNOTATION_PATH'], META_JSON_CUSTOM),    
}
    
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 1. Download TF Models Pretrained Models from Tensorflow Model Zoo and Install TFOD

In [4]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [5]:
issues = widgets.ToggleButton(value=False, description='lib issues')
verify = widgets.ToggleButton(value=False, description='Verification')
display(issues)
display(verify)

ToggleButton(value=False, description='lib issues')

ToggleButton(value=False, description='Verification')

In [6]:
if issues.value:# Install Tensorflow Object Detection 
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
    !pip install tensorflow --upgrade
    !pip uninstall protobuf matplotlib -y
    !pip install protobuf matplotlib==3.2
    
if verify.value:
    VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
    # Verify Installation
    !python {VERIFICATION_SCRIPT}
    
import object_detection

!wget {PRETRAINED_MODEL_URL}
!mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2021-11-22 10:39:30--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 2a00:1450:4001:829::2010, 142.250.186.144
Connecting to download.tensorflow.org (download.tensorflow.org)|2a00:1450:4001:829::2010|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19,56M  22,4MB/s    in 0,9s    

2021-11-22 10:39:31 (22,4 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x32

# 2. Create Label Map

In [7]:
labels = [{'name': objectChooser.value, 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')
        
with open(files['LABELTXT'], 'w') as f:
    for label in labels:
        f.write('{}\n'.format(label['name']))
        


# 3. Update Config For Transfer Learning

In [8]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format



In [9]:
# !pip install tensorflow-gpu==1.15

In [10]:
#!cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [11]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [12]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [13]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 4. Train the model

In [14]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

steps = widgets.IntText(
    value=2000,
    description='Trainsteps:',
    disabled=False
)
display(steps)


IntText(value=2000, description='Trainsteps:')

In [15]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'],steps.value)
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=pupil/models/ssd_mobnet_coco_pupil --pipeline_config_path=pupil/models/ssd_mobnet_coco_pupil/pipeline.config --num_train_steps=2000


In [16]:
!{command}

2021-11-22 10:39:48.678363: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-22 10:39:48.680722: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/daniel/miniconda3/envs/ML/lib/python3.7/site-packages/cv2/../../lib64:
2021-11-22 10:39:48.681093: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-22 10:39:48.681425: I tensorflow/core/platform/cpu_fe

# 5. Evaluate the Model

In [17]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=pupil/models/ssd_mobnet_coco_pupil --pipeline_config_path=pupil/models/ssd_mobnet_coco_pupil/pipeline.config --checkpoint_dir=pupil/models/ssd_mobnet_coco_pupil


In [18]:
!{command} # Has to be manually stopped after displaying the metrics

2021-11-22 10:40:53.085133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-22 10:40:53.087466: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/daniel/miniconda3/envs/ML/lib/python3.7/site-packages/cv2/../../lib64:
2021-11-22 10:40:53.087829: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
W1122 10:40:53.090115 139972595029824 model_lib_v2.py:1082] F

INFO:tensorflow:Performing evaluation on 18 images.
I1122 10:41:12.958970 139972595029824 coco_evaluation.py:293] Performing evaluation on 18 images.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
I1122 10:41:12.959126 139972595029824 coco_tools.py:116] Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.00s)
I1122 10:41:12.960454 139972595029824 coco_tools.py:138] DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.644
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.679
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.577
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100

# 6. Conversion to TFLite

In [19]:
TFLITE_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_tflite_graph_tf2.py ')
command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT ,files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'], paths['TFLITE_PATH'])
print(command)

python Tensorflow/models/research/object_detection/export_tflite_graph_tf2.py  --pipeline_config_path=pupil/models/ssd_mobnet_coco_pupil/pipeline.config --trained_checkpoint_dir=pupil/models/ssd_mobnet_coco_pupil --output_directory=pupil/models/ssd_mobnet_coco_pupil/tfliteexport


In [20]:
!{command}

2021-11-22 10:42:10.912632: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-22 10:42:10.915002: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2021-11-22 10:42:10.915360: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-22 10:42:10.919620: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library

2021-11-22 10:42:22.122133: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
W1122 10:42:31.964796 140553496201024 save.py:254] Found untraced functions such as WeightSharedConvolutionalBoxPredictor_layer_call_fn, WeightSharedConvolutionalBoxPredictor_layer_call_and_return_conditional_losses, WeightSharedConvolutionalBoxHead_layer_call_fn, WeightSharedConvolutionalBoxHead_layer_call_and_return_conditional_losses, WeightSharedConvolutionalBoxPredictor_layer_call_fn while saving (showing 5 of 260). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: pupil/models/ssd_mobnet_coco_pupil/tfliteexport/saved_model/assets
I1122 10:42:34.829211 140553496201024 builder_impl.py:781] Assets written to: pupil/models/ssd_mobnet_coco_pupil/tfliteexport/saved_model/assets


In [21]:
FROZEN_TFLITE_PATH = os.path.join(paths['TFLITE_PATH'], 'saved_model')
TFLITE_MODEL = os.path.join(paths['TFLITE_PATH'], 'saved_model', objectChooser.value + '.tflite')

In [22]:
command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--input_shapes=1,320,320,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
--inference_type=FLOAT \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL, )
print(command)

tflite_convert --saved_model_dir=pupil/models/ssd_mobnet_coco_pupil/tfliteexport/saved_model --output_file=pupil/models/ssd_mobnet_coco_pupil/tfliteexport/saved_model/pupil.tflite --input_shapes=1,320,320,3 --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --inference_type=FLOAT --allow_custom_ops


In [23]:
!{command}

2021-11-22 10:42:42.757947: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-22 10:42:42.760448: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2021-11-22 10:42:42.760798: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-22 10:42:43.197188: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library